# Examen 2 RI
- Angel Falcon

# Librerias

In [135]:
from sentence_transformers import SentenceTransformer, CrossEncoder
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import ndcg_score
import pandas as pd
from tqdm import tqdm
import faiss
import kagglehub
import json

In [136]:
# Preprocesamiento
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import word_tokenize

In [137]:
# Descargar recursos NLTK necesarios
recursos_nltk = ['punkt', 'punkt_tab', 'stopwords', 'wordnet']
for recurso in recursos_nltk:
    try:
        if recurso in ['punkt', 'punkt_tab']:
            nltk.data.find(f'tokenizers/{recurso}')
        else:
            nltk.data.find(f'corpora/{recurso}')
    except LookupError:
        print(f"Descargando {recurso}...")
        nltk.download(recurso, quiet=True)


Descargando wordnet...


# Parte 1: Carga del Dataset

In [138]:
def cargar_arxiv_dataset(limite=10000):
    """Carga el dataset de  Kaggle"""
    print("="*80)
    print("CARGANDO DATASET ")
    print("="*80)

    # Descargar dataset
    print("\n Descargando dataset de arXiv desde Kaggle...")
    path = kagglehub.dataset_download("Cornell-University/arxiv")
    print(f" Dataset descargado en: {path}")

    # Cargar metadatos (JSON Line format)
    print(f"\n Cargando {limite:,} artículos científicos...")
    metadata = []

    with open(f"{path}/arxiv-metadata-oai-snapshot.json", 'r') as f:
        for i, line in enumerate(tqdm(f, desc="Cargando", total=limite)):
            if i >= limite:
                break
            try:
                metadata.append(json.loads(line))
            except json.JSONDecodeError:
                continue

    df_arxiv = pd.DataFrame(metadata)
    print(f"\n Dataset cargado: {len(df_arxiv):,} artículos científicos")

    return df_arxiv, path
cargar_arxiv_dataset()


CARGANDO DATASET 

 Descargando dataset de arXiv desde Kaggle...
Using Colab cache for faster access to the 'arxiv' dataset.
 Dataset descargado en: /kaggle/input/arxiv

 Cargando 10,000 artículos científicos...


Cargando: 100%|██████████| 10000/10000 [00:00<00:00, 55087.24it/s]



 Dataset cargado: 10,000 artículos científicos


(             id             submitter  \
 0     0704.0001        Pavel Nadolsky   
 1     0704.0002          Louis Theran   
 2     0704.0003           Hongjun Pan   
 3     0704.0004          David Callan   
 4     0704.0005    Alberto Torchinsky   
 ...         ...                   ...   
 9995  0706.1309  E. V. Sampathkumaran   
 9996  0706.1310       Susanne Reffert   
 9997  0706.1311          Max Lieblich   
 9998  0706.1312          Labib Haddad   
 9999  0706.1313      Thierry Coulbois   
 
                                                 authors  \
 0     C. Bal\'azs, E. L. Berger, P. M. Nadolsky, C.-...   
 1                       Ileana Streinu and Louis Theran   
 2                                           Hongjun Pan   
 3                                          David Callan   
 4              Wael Abu-Shammala and Alberto Torchinsky   
 ...                                                 ...   
 9995              Kartik K Iyer and E.V. Sampathkumaran   
 9996         

In [139]:
def explorar_dataset_arxiv(df_arxiv):
    """Explora y muestra información del dataset"""
    print("INFORMACIÓN DEL DATASET")

    print(f"\n Shape: {df_arxiv.shape[0]:,} filas x {df_arxiv.shape[1]} columnas")

    print("\n Columnas disponibles:")
    for i, col in enumerate(df_arxiv.columns, 1):
        print(f"  {i:2d}. {col}")


    if 'categories' in df_arxiv.columns:
        print(f"\n  • Top 10 categorías:")
        print(df_arxiv['categories'].value_counts().head(10))
explorar_dataset_arxiv(df_arxiv)

INFORMACIÓN DEL DATASET

 Shape: 10,000 filas x 15 columnas

 Columnas disponibles:
   1. id
   2. submitter
   3. authors
   4. title
   5. comments
   6. journal-ref
   7. doi
   8. report-no
   9. categories
  10. license
  11. abstract
  12. versions
  13. update_date
  14. authors_parsed
  15. abstract_length

  • Top 10 categorías:
categories
astro-ph              1718
hep-ph                 475
quant-ph               466
hep-th                 427
gr-qc                  218
cond-mat.mtrl-sci      194
cond-mat.mes-hall      179
cond-mat.str-el        164
cond-mat.stat-mech     145
nucl-th                141
Name: count, dtype: int64


In [140]:
def preparar_dataset_arxiv(df_arxiv):
    """Prepara el dataset para recuperación de información"""
    print("PREPARACIÓN DEL DATASET")

    # Limpiar datos
    print("\n Limpiando datos...")
    antes = len(df_arxiv)

    # Eliminar registros sin abstract
    df_arxiv = df_arxiv[df_arxiv['abstract'].notna()]
    df_arxiv = df_arxiv[df_arxiv['abstract'].astype(str).str.strip() != '']

    # Eliminar registros sin título
    df_arxiv = df_arxiv[df_arxiv['title'].notna()]
    df_arxiv = df_arxiv[df_arxiv['title'].astype(str).str.strip() != '']

    despues = len(df_arxiv)
    print(f"Registros válidos: {despues:,}")

    # Crear columna de texto completo (título + abstract)
    print("\nCreando columna de texto completo...")
    df_arxiv['texto_completo'] = (
        df_arxiv['title'].astype(str) + ' ' + df_arxiv['abstract'].astype(str)
    )

    # Crear columna de ID simple si no existe
    if 'id' not in df_arxiv.columns:
        df_arxiv['doc_id'] = df_arxiv.index.astype(str)
    else:
        df_arxiv['doc_id'] = df_arxiv['id'].astype(str)

    # Resetear índice
    df_arxiv = df_arxiv.reset_index(drop=True)

    print(f" Dataset preparado: {len(df_arxiv):,} artículos listos")

    return df_arxiv
preparar_dataset_arxiv(df_arxiv)

PREPARACIÓN DEL DATASET

 Limpiando datos...
Registros válidos: 10,000

Creando columna de texto completo...
 Dataset preparado: 10,000 artículos listos


,id,submitter,authors,title,comments,journal-ref,doi,report-no,categories,license,abstract,versions,update_date,authors_parsed,abstract_length,texto_completo,doc_id
0,0704.0001,Pavel Nadolsky,"C. Bal\'azs, E. L. Berger, P. M. Nadolsky, C.-...",Calculation of prompt diphoton production cros...,"37 pages, 15 figures; published version","Phys.Rev.D76:013009,2007",10.1103/PhysRevD.76.013009,ANL-HEP-PR-07-12,hep-ph,None,A fully differential calculation in perturba...,"[{'version': 'v1', 'created': 'Mon, 2 Apr 2007...",2008-11-26,"[[Balázs, C., ], [Berger, E. L., ], [Nadolsky,...",983,Calculation of prompt diphoton production cros...,0704.0001
1,0704.0002,Louis Theran,Ileana Streinu and Louis Theran,Sparsity-certifying Graph Decompositions,To appear in Graphs and Combinatorics,None,None,None,math.CO cs.CG,http://arxiv.org/licenses/nonexclusive-distrib...,"We describe a new algorithm, the $(k,\ell)$-...","[{'version': 'v1', 'created': 'Sat, 31 Mar 200...",2008-12-13,"[[Streinu, Ileana, ], [Theran, Louis, ]]",798,Sparsity-certifying Graph Decompositions We ...,0704.0002
2,0704.0003,Hongjun Pan,Hongjun Pan,The evolution of the Earth-Moon system based o...,"23 pages, 3 figures",None,None,None,physics.gen-ph,None,The evolution of Earth-Moon system is descri...,"[{'version': 'v1', 'created': 'Sun, 1 Apr 2007...",2008-01-13,"[[Pan, Hongjun, ]]",880,The evolution of the Earth-Moon system based o...,0704.0003
3,0704.0004,David Callan,David Callan,A determinant of Stirling cycle numbers counts...,11 pages,None,None,None,math.CO,None,We show that a determinant of Stirling cycle...,"[{'version': 'v1', 'created': 'Sat, 31 Mar 200...",2007-05-23,"[[Callan, David, ]]",248,A determinant of Stirling cycle numbers counts...,0704.0004
4,0704.0005,Alberto Torchinsky,Wael Abu-Shammala and Alberto Torchinsky,From dyadic $\Lambda_{\alpha}$ to $\Lambda_{\a...,None,"Illinois J. Math. 52 (2008) no.2, 681-689",None,None,math.CA math.FA,None,In this paper we show how to compute the $\L...,"[{'version': 'v1', 'created': 'Mon, 2 Apr 2007...",2013-10-15,"[[Abu-Shammala, Wael, ], [Torchinsky, Alberto, ]]",223,From dyadic $\Lambda_{\alpha}$ to $\Lambda_{\a...,0704.0005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0706.1309,E. V. Sampathkumaran,Kartik K Iyer and E.V. Sampathkumaran,Superconducting behavior of the solid solution...,"Physica C, in press",None,10.1016/j.physc.2007.05.049,None,cond-mat.supr-con cond-mat.str-el,None,The compound Y2PdGe3 was earlier reported by...,"[{'version': 'v1', 'created': 'Sat, 9 Jun 2007...",2009-11-13,"[[Iyer, Kartik K, ], [Sampathkumaran, E. V., ]]",684,Superconducting behavior of the solid solution...,0706.1309
9996,0706.1310,Susanne Reffert,S. Reffert,The Geometer's Toolkit to String Compactificat...,Lecture notes based on lectures given at the W...,None,None,None,hep-th,None,These lecture notes are meant to serve as an...,"[{'version': 'v1', 'created': 'Sat, 9 Jun 2007...",2009-09-29,"[[Reffert, S., ]]",441,The Geometer's Toolkit to String Compactificat...,0706.1310
9997,0706.1311,Max Lieblich,Max Lieblich,Compactified moduli of projective bundles,28 pages. Major reogranization and clarificati...,"Algebra Number Theory 3 (2009), no. 6, 653-695",None,None,math.AG math.RT,http://arxiv.org/licenses/nonexclusive-distrib...,We present a method for compactifying stacks...,"[{'version': 'v1', 'created': 'Mon, 11 Jun 200...",2018-06-18,"[[Lieblich, Max, ]]",507,Compactified moduli of projective bundles We...,0706.1311
9998,0706.1312,Labib Haddad,Barben-Jean Coffi-Nketsia and Labib Haddad,Entrelacement d'alg\`ebres de Lie [Wreath prod...,A moderately detailed english summary of the p...,None,None,None,math.RT,None,Full details are given for the definition an...,"[{'version': 'v1', 'created': 'Sat, 9 Jun 2007...",2007-06-12,"[[Coffi-Nketsia, Barben-Jean, ], [Haddad, Labi...",1153,Entrelacement d'alg\`ebres de Lie [Wreath prod...,0706.1312


# Parte 2 : Preprocesamiento de Datos

In [141]:
def inicializar_preprocesamiento():
    """Inicializar herramientas de preprocesamiento"""
    return {
        'stemmer': PorterStemmer(),
        'lemmatizer': WordNetLemmatizer(),
        'stop_words': set(stopwords.words('english'))
    }


def tokenizar(documento):
    """Tokenización del texto"""
    doc_tokenizacion = word_tokenize(documento.lower())
    return doc_tokenizacion


def normalizar(tokenizacion):
    """Normalización: remover puntuación y caracteres especiales"""
    doc_normalizado = [token for token in tokenizacion if token.isalpha()]
    return doc_normalizado


def remover_stopwords(tokenizacion, stop_words):
    """Remoción de stopwords"""
    doc_sin_stopwords = [token for token in tokenizacion if token not in stop_words]
    return doc_sin_stopwords


def stemming(tokenizacion, stemmer):
    """Aplicación de stemming (reducción a raíz)"""
    doc_stemmed = [stemmer.stem(token) for token in tokenizacion]
    return doc_stemmed


def lematizar(tokenizacion, lemmatizer):
    """Aplicación de lematización"""
    doc_lemmatized = [lemmatizer.lemmatize(token) for token in tokenizacion]
    return doc_lemmatized


def procesamiento_doc(documento, preprocesamiento_tools, usar_lemmatizacion=False):
    """Pipeline completo de procesamiento"""
    # 1. Tokenización
    doc_tokenizacion = tokenizar(documento)

    # 2. Normalización
    doc_normalizacion = normalizar(doc_tokenizacion)

    # 3. Remover stopwords
    doc_sin_stopwords = remover_stopwords(doc_normalizacion, preprocesamiento_tools['stop_words'])

    # 4. Stemming o Lematización
    if usar_lemmatizacion:
        doc_procesado = lematizar(doc_sin_stopwords, preprocesamiento_tools['lemmatizer'])
    else:
        doc_procesado = stemming(doc_sin_stopwords, preprocesamiento_tools['stemmer'])

    return doc_procesado


def preprocesar_texto(texto, preprocesamiento_tools, usar_lemmatizacion=False):
    """Preprocesa texto y lo devuelve como string"""
    tokens = procesamiento_doc(texto, preprocesamiento_tools, usar_lemmatizacion)
    return ' '.join(tokens)


def preprocesar_documentos(df_arxiv, preprocesamiento_tools, columna='texto_completo'):
    """Preprocesa todos los documentos del dataset"""
    print("\n Preprocesando documentos...")

    documentos_procesados = []
    for texto in tqdm(df_arxiv[columna], desc="Preprocesando"):
        texto_procesado = preprocesar_texto(str(texto), preprocesamiento_tools)
        documentos_procesados.append(texto_procesado)

    print(f" {len(documentos_procesados):,} documentos preprocesados")

    return documentos_procesados

# Parte 3: Embeddings

In [142]:
def cargar_modelo_embeddings(modelo='all-MiniLM-L6-v2'):
    """Carga un modelo de sentence-transformers"""
    print(f"\n Cargando modelo de embeddings: {modelo}")
    model = SentenceTransformer(modelo)
    return model


def generar_embeddings_texto(documentos, modelo, show_progress=True):
    """Genera embeddings para una lista de documentos"""
    print("\n Generando embeddings...")
    embeddings = modelo.encode(documentos, show_progress_bar=show_progress, batch_size=32)
    embeddings = np.array(embeddings)
    print(f" Embeddings generados: {embeddings.shape}")
    return embeddings

# Parte 4: Recuperación Inicial Con Faiss

In [143]:
def crear_indice_faiss(embeddings):
    """Crea un índice FAISS para búsqueda vectorial eficiente"""
    print("\n Creando índice FAISS...")
    dimension = embeddings.shape[1]

    # Normalizar embeddings para usar inner product como similitud coseno
    embeddings_normalized = embeddings.copy().astype('float32')
    faiss.normalize_L2(embeddings_normalized)

    # Crear índice FAISS (IndexFlatIP para Inner Product)
    index = faiss.IndexFlatIP(dimension)
    index.add(embeddings_normalized)

    print(f" Índice FAISS creado: {index.ntotal:,} vectores de dimensión {dimension}")
    return index


def buscar_con_faiss(query_embedding, index, top_k=10):
    """Busca documentos similares usando FAISS"""
    # Normalizar query embedding
    query_embedding = query_embedding.reshape(1, -1).astype('float32')
    faiss.normalize_L2(query_embedding)

    # Búsqueda en FAISS
    similarities, indices = index.search(query_embedding, top_k)

    return indices[0], similarities[0]


def buscar_similares_faiss(query, model, index, df_arxiv, top_k=10):
    """Busca artículos similares usando FAISS"""
    # Generar embedding de la query
    query_embedding = model.encode(query)

    # Buscar con FAISS
    indices, similarities = buscar_con_faiss(query_embedding, index, top_k)

    # Construir resultados
    resultados = []
    for idx, sim in zip(indices, similarities):
        resultados.append({
            'indice': int(idx),
            'similitud': float(sim),
            'doc_id': df_arxiv.iloc[idx]['doc_id'],
            'title': df_arxiv.iloc[idx]['title'],
            'abstract': df_arxiv.iloc[idx]['abstract'],
            'categories': df_arxiv.iloc[idx].get('categories', 'N/A'),
            'documento': df_arxiv.iloc[idx]['texto_completo']
        })

    return resultados
    crear_indice_faiss()

# Parte 5: Re-Ranking

In [144]:
def cargar_modelo_reranker(modelo='cross-encoder/ms-marco-MiniLM-L-6-v2'):
    """Carga un modelo de reranking (Cross-Encoder)"""
    print(f"\n Cargando modelo de re-ranking: {modelo}")
    cross_encoder = CrossEncoder(modelo)
    return cross_encoder


def aplicar_reranking(query, resultados, cross_encoder, top_k=10):
    """Aplica re-ranking usando Cross-Encoder"""
    print(" Aplicando re-ranking...")

    # Preparar pares query-documento
    pares = [[query, r['abstract']] for r in resultados]

    # Obtener scores del cross-encoder
    scores = cross_encoder.predict(pares)

    # Actualizar scores y reordenar
    for i, r in enumerate(resultados):
        r['score_reranking'] = float(scores[i])

    # Ordenar por score de re-ranking
    resultados_reranked = sorted(resultados, key=lambda x: x['score_reranking'], reverse=True)

    return resultados_reranked[:top_k]

# Parte 6: Busqueda Consultas

In [145]:
def buscar(query, model, index, df_arxiv, top_k=10,
           cross_encoder=None, aplicar_reranking_flag=False):
    """
    Realiza una búsqueda completa
    """
    # Recuperación inicial con FAISS
    k_inicial = top_k if not aplicar_reranking_flag else top_k * 2
    resultados = buscar_similares_faiss(query, model, index, df_arxiv, top_k=k_inicial)

    # Aplicar re-ranking si se solicita
    if aplicar_reranking_flag:
        if cross_encoder is None:
            cross_encoder = cargar_modelo_reranker()

        resultados = aplicar_reranking(query, resultados, cross_encoder, top_k)

    return resultados


def mostrar_resultados_busqueda(resultados, titulo="Resultados", mostrar_abstract=True, max_chars=300):
    """Muestra resultados de búsqueda de forma formateada"""
    print("\n" + "="*80)
    print(f"{titulo} - Top {len(resultados)}")
    print("="*80)

    for i, r in enumerate(resultados, 1):
        score_key = 'score_reranking' if 'score_reranking' in r else 'similitud'
        score = r[score_key]

        print(f"\n{i}. Doc ID: {r['doc_id']}")
        print(f"   Título: {r['title']}")
        print(f"   Categoría: {r.get('categories', 'N/A')}")
        print(f"   Score: {score:.4f}")

        if mostrar_abstract:
            abstract = r.get('abstract', '')
            if len(abstract) > max_chars:
                abstract = abstract[:max_chars] + "..."
            print(f"   Abstract: {abstract}")

    print("\n" + "="*80)


def ejecutar_consultas_ejemplo(queries_ejemplo, model, index, df_arxiv,
                               cross_encoder=None, top_k=5):
    """Ejecuta múltiples consultas de ejemplo"""
    print(" CONSULTAS")

    for query in queries_ejemplo:
        print(f"\n{'='*80}")
        print(f"Query: {query}")
        print(f"{'='*80}")

        # Recuperación inicial
        print("\n RECUPERACIÓN INICIAL (FAISS):")
        resultados_inicial = buscar(query, model, index, df_arxiv,
                                    top_k=top_k, aplicar_reranking_flag=False)
        mostrar_resultados_busqueda(resultados_inicial, "Resultados Iniciales",
                                   mostrar_abstract=True, max_chars=200)

        # Re-ranking
        print("\n DESPUÉS DE RE-RANKING:")
        resultados_rerank = buscar(query, model, index, df_arxiv,
                                   top_k=top_k, cross_encoder=cross_encoder,
                                   aplicar_reranking_flag=True)
        mostrar_resultados_busqueda(resultados_rerank, "Resultados Re-rankeados",
                                   mostrar_abstract=True, max_chars=200)

# Parte 7: Queries

In [149]:
def crear_queries_ejemplo():
    """Crea queries de ejemplo para el dataset de arXiv"""
    queries = {
        'q1': 'International conflict',
        'q2': 'Economic policy news',
        'q3': 'Natural disasters',

    }
    return queries


def evaluar_relevancia_manual(resultados, query):

    print(f"\nEvaluación manual para query: '{query}'")
    print("Por favor evalúa la relevancia de cada resultado (0=no relevante, 1=relevante, 2=muy relevante)")

    relevancia = []
    for i, r in enumerate(resultados, 1):
        print(f"\n{i}. {r['title']}")
        # En un caso real, aquí pedirías input del usuario
        # Para demostración, asumimos relevancia aleatoria
        rel = np.random.choice([0, 1, 2])
        relevancia.append(rel)

    return relevancia

# Parte 8: Sistema

In [ ]:
# 1. Cargar dataset
df_arxiv, path = cargar_arxiv_dataset(limite=5000)

# 2. Explorar dataset
explorar_dataset_arxiv(df_arxiv)

# 3. Preparar dataset
df_arxiv = preparar_dataset_arxiv(df_arxiv)



In [ ]:
# 4. Preprocesamiento
print("PREPROCESAMIENTO DE DATOS")
preprocesamiento_tools = inicializar_preprocesamiento()
documentos_procesados = preprocesar_documentos(
    df_arxiv, preprocesamiento_tools, columna='texto_completo'
)

# 5. Embeddings
print("GENERANDO EMBEDDINGS")
model = cargar_modelo_embeddings('all-MiniLM-L6-v2')
embeddings = generar_embeddings_texto(documentos_procesados, model)



In [154]:
# 6. Crear índice FAISS
print("CREANDO ÍNDICE FAISS")
index = crear_indice_faiss(embeddings)

# 7. Cargar modelo de re-ranking
print("CARGANDO CROSS-ENCODER")
cross_encoder = cargar_modelo_reranker()



CREANDO ÍNDICE FAISS

 Creando índice FAISS...
 Índice FAISS creado: 5,000 vectores de dimensión 384
CARGANDO CROSS-ENCODER

 Cargando modelo de re-ranking: cross-encoder/ms-marco-MiniLM-L-6-v2


In [155]:
# 8. Crear queries de ejemplo
queries = crear_queries_ejemplo()
queries_lista = list(queries.values())[:3]

# 9. Ejecutar consultas
print("EJECUTANDO CONSULTAS")
ejecutar_consultas_ejemplo(queries_lista, model, index, df_arxiv, cross_encoder, top_k=5)


EJECUTANDO CONSULTAS
 CONSULTAS

Query: International conflict

 RECUPERACIÓN INICIAL (FAISS):

Resultados Iniciales - Top 5

1. Doc ID: 0704.3862
   Título: An Integrated Human-Computer System for Controlling Interstate Disputes
   Categoría: stat.AP
   Score: 0.2704
   Abstract:   In this paper we develop a scientific approach to control inter-country
conflict. This system makes use of a neural network and a feedback control
approach. It was found that by controlling the four...

2. Doc ID: 0704.2883
   Título: A physicist's view of the notion of "racism"
   Categoría: physics.soc-ph
   Score: 0.2617
   Abstract:   It is not uncommon, e.g. in the media, that specific groups are categorized
as being racist. Based on an extensive dataset of intermarriage statistics our
study questions the legitimacy of such char...

3. Doc ID: 0704.1270
   Título: Core-Corona Separation in Ultra-Relativistic Heavy Ion Collisions
   Categoría: nucl-th astro-ph hep-ph nucl-ex
   Score: 0.2368
   Abstract

# Análisis de Resultados
 - Discusión sobre la calidad de los resultados obtenidos.


Los resultados mejora notablemente cuando se aplica el re-ranking con el modelo Cross-Encoder en esta fase, el sistema analiza de forma más detallada la relación entre la consulta y cada documento.

 - Comparación entre los resultados de la recuperación inicial y el ranking final.

 Al comparar la recuperación inicial con el ranking final, se observa que la recuperación inicial es más rápida y amplia, pero menos precisa, ya que prioriza similitud vectorial. En cambio el ranking final es más lento, pero mucho más preciso.
